In [32]:
!pip install PyPortfolioOpt
!pip install portfolio-backtest
!pip install riskfolio-lib
!pip install yesg

Now that our backtest has been carried out and confirmed the stability of our portfolio's performance over time, we will now show how, once we have obtained the weights of our risk parity portfolio on the ETFs, we can extract the weights of the stocks that make them up.
We illustrate this with the weights of the ERC trained over the period January to September 2024 and we will calculate the weights that we will hold over the October, November and December quarters.

In [ ]:
from GeneralFunction import download_data,calculate_metrics,split_data,weight_dic,transaction_costs,objective,optimize_risk_par,optimize_min_var,add_colored_traces
import yfinance as yf
import certifi
import requests
from io import BytesIO, StringIO
import pandas as pd
import yesg
import warnings
warnings.filterwarnings("ignore")

In [36]:
params = {
        "i": 63 ,
        "j": 189,
        "k": 63,
        "start_date": "2024-01-01",
        "end_date": "2024-06-06",
        "tickers": ['SPHQ', 'IVE', 'SPYD', 'SPLV', 'SPMO']
        }


returns = download_data(params["tickers"], params["start_date"], params["end_date"])
weights = optimize_risk_par(returns)

[*********************100%***********************]  5 of 5 completed


In [37]:
def download_data(url):
    """
    Télécharge et retourne les données d'une URL, en gérant les fichiers Excel et CSV de manière appropriée.
    """
    if url.endswith('.xls') or url.endswith('.xlsx'):
        skip_rows = 4
        response = requests.get(url)
        response.raise_for_status()
        data = pd.read_excel(BytesIO(response.content), skiprows=skip_rows)
    else:
        skip_rows = 0
        response = requests.get(url)
        response.raise_for_status()
        csv_data = response.content.decode('utf-8')
        try:
            data = pd.read_csv(StringIO(csv_data), skiprows=skip_rows)
        except pd.errors.ParserError:
            try:
                data = pd.read_csv(StringIO(csv_data), delimiter=';', skiprows=skip_rows)
            except pd.errors.ParserError:
                data = pd.read_csv(StringIO(csv_data), skiprows=9 + skip_rows)

    if not any('weight' in col.lower() for col in data.columns):
        data = pd.read_csv(StringIO(csv_data), skiprows=9 + skip_rows)
    return data

def normalize_ticker(ticker):
    """
    Normalise le ticker en supprimant les espaces et les caractères inutiles.
    """
    if pd.isna(ticker):  # Vérifier si ticker est NaN
        return ''
    return ticker.replace('/', '').strip().upper()

def calculate_total_weights_per_etf(url, etf_weight):
    """
    Calcule le poids total pour chaque action d'un fichier CSV ou Excel.
    """
    data = download_data(url)
    weight_column = next((col for col in data.columns if 'weight' in col.lower()), None)
    if weight_column is None:
        print("Colonne 'Weight' ou 'Weight (%)' introuvable dans le fichier.")
        return None

    holding_ticker_column = next((col for col in data.columns if 'holding ticker' in col.lower()), None)
    ticker_column = next((col for col in data.columns if 'ticker' in col.lower()), None)
    if holding_ticker_column:
        ticker_column = holding_ticker_column
    elif not ticker_column:
        print("Aucune colonne 'Ticker' ou 'Holding Ticker' introuvable dans le fichier.")
        return None

    data['Weighted_Weight'] = data[weight_column] * etf_weight

    if 'Name' in data.columns:
        data = data[~data['Name'].isin(["US DOLLAR", "Cash/Receivables/Payables", "S+P500 EMINI FUT  DEC24", "CASH COLLATERAL MSFUT USD", "USD CASH", "SSI US GOV MONEY MARKET CLASS"])]

    # Normaliser les tickers
    data[ticker_column] = data[ticker_column].apply(normalize_ticker)

    total_weights = data.groupby(ticker_column)[['Name', 'Weighted_Weight']].sum().reset_index()
    total_weights.columns = ['Ticker', 'Name', 'Total_Weight']
    total_weights = total_weights[total_weights['Total_Weight'] > 0]
    return total_weights

def calculate_total_weights(weights):
    # Téléchargement et traitement des données pour chaque URL
    urls = [
        "https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=SPHQ",
        "https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=SPLV",
        "https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=SPMO",
        "https://www.ishares.com/us/products/239728/ishares-sp-500-value-etf/1467271812596.ajax?fileType=csv&fileName=IVE_holdings&dataType=fund",
        "https://www.ssga.com/us/en/intermediary/library-content/products/fund-data/etfs/us/holdings-daily-us-en-spyd.xlsx"
    ]

    final_data = pd.DataFrame()

    # Appliquer chaque URL et poids d'ETF
    for url, weight in zip(urls, weights):
        etf_data = calculate_total_weights_per_etf(url, weight)
        if etf_data is not None:
            final_data = pd.concat([final_data, etf_data])

    # Étape finale : Regrouper par ticker exact et sommer les poids
    final_data = final_data.groupby('Ticker', as_index=False)['Total_Weight'].sum()
    final_data['Total_Weight'] /= final_data['Total_Weight'].sum()

    # Trier les données en ordre décroissant de poids
    #final_data.sort_values(by='Total_Weight', ascending=False, inplace=True)

    # Vérification des doublons
    duplicate_tickers = final_data[final_data.duplicated(subset='Ticker', keep=False)]
    if not duplicate_tickers.empty:
        print("Doublons trouvés dans les tickers :")
        print(duplicate_tickers)
    #else:
    #    print("Aucun doublon trouvé dans les tickers.")

    # Affichage du résultat final
    #print(final_data.to_string(index=False))

    # Enregistrer dans un fichier CSV
    #final_data.to_csv("poids_actions_etf.csv", index=False)
    #print("Les données complètes ont été enregistrées dans 'poids_actions_etf.csv'.")

    final_data.columns = ['Symbol', 'Weights']
    final_data = final_data.set_index('Symbol')

    return final_data

#Stocks with several classes: fox and news corp

In [38]:
import yfinance as yf
import pandas as pd

def calculate_portfolio_value(weights: pd.DataFrame):
    """
    Calculate the weighted value of stocks in a portfolio using Yahoo Finance data.

    Args:
        weights (pd.DataFrame): A DataFrame indexed by stock symbols, with a 'Weights' column.

    Returns:
        pd.DataFrame: A DataFrame with columns ['Symbol', 'Weight', 'Last Close', 'Weighted Value'].
    """
    # Ensure the DataFrame has the expected structure
    if 'Weights' not in weights.columns:
        raise ValueError("The input DataFrame must have a 'Weights' column.")

    results = []

    for symbol, row in weights.iterrows():
        weight = row['Weights']

        # Fetch the stock data from Yahoo Finance
        try:
            stock = yf.Ticker(symbol)
            last_close = stock.history(period='1d')['Close'].iloc[-1]

            # Calculate the weighted value
            weighted_value = weight * last_close
            results.append({'Symbol': symbol, 'Weight': weight, 'Last Close': last_close, 'Weighted Value': weighted_value})
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")

    # Return results as a DataFrame
    return pd.DataFrame(results)


In [39]:
stock_weights = calculate_total_weights(weights)
stock_weights.sort_values(by='Weights', ascending=False)

,Weights
Symbol,
NVDA,0.030750
AMZN,0.029636
AAPL,0.027115
META,0.023787
BRKB,0.018218
...,...
PNR,0.000037
ROL,0.000037
GNRC,0.000019


In [40]:
calculate_portfolio_value(stock_weights)

ERROR:yfinance:$BFB: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Error fetching data for BFB: single positional indexer is out-of-bounds


ERROR:yfinance:$BRKB: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Error fetching data for BRKB: single positional indexer is out-of-bounds


ERROR:yfinance:Could not get exchangeTimezoneName for ticker 'CTAS' reason: 'chart'
ERROR:yfinance:$CTAS: possibly delisted; no price data found  (period=1d)


Error fetching data for CTAS: single positional indexer is out-of-bounds


ERROR:yfinance:$XTSLA: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Error fetching data for XTSLA: single positional indexer is out-of-bounds


,Symbol,Weight,Last Close,Weighted Value
0,A,0.001090,134.509995,0.146642
1,AAPL,0.027115,254.490005,6.900543
2,ABBV,0.003709,175.580002,0.651290
3,ABT,0.003497,114.230003,0.399469
4,ACGL,0.000793,90.989998,0.072169
...,...,...,...,...
460,XYL,0.000224,117.139999,0.026278
461,YUM,0.002353,132.360001,0.311505
462,ZBH,0.000168,107.120003,0.018023
463,ZBRA,0.000093,393.040009,0.036738


### Constraints
- We do not take into account the worst stocks in terms of ESG scores
- Sector constraints (the s&p is already concentrated so it might be interesting to allow a difference with the sector wiehgts in the original s&p 500)
- Concentration limits (a limit for each stock)
- Liquidity constraints (use only stocks with volume higher than a threshold): not so interesting because all s&p 500 stocks are liquid
no short allowed
- tracking error: determined with backtest

In [41]:
def esg_constraints(stock_weights, quantile_threshold):
    """
    quantile_threshold: float between 0 and 1 to keep the best quantile_threshold quantile of the stocks
    stock_weights: DataFrame with the weights of the stocks in the portfolio
    """
    # get the s&p 500 tickers with their name, sector and sub-industry
    df_sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0][['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry']]
    df_sp500 = df_sp500.set_index('Symbol') # set the index to be the symbol

    df_sp500.drop(index='GOOG', inplace=True) # drop the GOOG row because there is already a GOOGL row

    df_esg = df_sp500.copy()
    for ticker in df_sp500.index:
        try:
            df_esg.loc[ticker, "ESG Score"] = yesg.get_historic_esg(ticker).iloc[-1,0]
        except AttributeError:
            pass

    #drop all the stocks that have no ESG score
    df_esg.dropna(axis=0, inplace=True)

    #The best ESG score is 0
    #drop the worst stocks in terms of ESG score
    df_esg = df_esg[df_esg['ESG Score'] < df_esg['ESG Score'].quantile(quantile_threshold)]

    #drop all the stocks that have an ESG score above the threshold
    #threshold = 30
    #df_esg = df_esg[df_esg['ESG Score'] < threshold]

    df_esg = df_esg.merge(stock_weights, left_index=True, right_index=True)

    df_esg['Weights'] /= df_esg['Weights'].sum()

    return df_esg

In [42]:
quantile_threshold = 0.9
df_esg = esg_constraints(stock_weights, quantile_threshold)
df_esg.sort_values(by='Weights', ascending=False)

An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.


,Security,GICS Sector,GICS Sub-Industry,ESG Score,Weights
Symbol,,,,,
NVDA,Nvidia,Information Technology,Semiconductors,12.23,0.034653
AMZN,Amazon,Consumer Discretionary,Broadline Retail,29.01,0.033398
AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",16.79,0.030557
COST,Costco,Consumer Staples,Consumer Staples Merchandise Retail,29.12,0.019084
AVGO,Broadcom,Information Technology,Semiconductors,19.20,0.017354
...,...,...,...,...,...
ROL,"Rollins, Inc.",Industrials,Environmental & Facilities Services,18.60,0.000042
ALLE,Allegion,Industrials,Building Products,19.83,0.000042
GNRC,Generac,Industrials,Electrical Components & Equipment,21.93,0.000021


## Sector constraints

In [43]:
def sector_constraints(df, sector_threshold):
    """
    df: DataFrame following the same format as the one returned by esg_constraints
    sector_threshold: float between 0 and 1 that a sector cannot exceed in the portfolio
    """
    #We calculate the weights of the stocks in each sector
    df_weights_by_sector = df[['GICS Sector', 'Weights']].groupby('GICS Sector').sum()

    #We create a list of the sectors that have a weight above the threshold
    sectors_above_threshold = df_weights_by_sector.loc[df_weights_by_sector['Weights'] > sector_threshold].index
    while len(sectors_above_threshold) > 0:
        for sector in sectors_above_threshold:
            #for each stock in the sector, we apply a factor such that the sum of all stocks in this sector is equal to the threshold
            df_sector = df[df['GICS Sector'] == sector]
            factor = sector_threshold / df_weights_by_sector.loc[sector, 'Weights']
            df.loc[df_sector.index, 'Weights'] = df_sector['Weights'] * factor

        #we increase the weights of the stocks in the other sectors to keep the sum of the weights equal to 1
        df.loc[~df['GICS Sector'].isin(sectors_above_threshold), "Weights"] /= df.loc[~df['GICS Sector'].isin(sectors_above_threshold), "Weights"].sum() / (1-sector_threshold*len(sectors_above_threshold))

        #Some sectors may now have a weight above the threshold so we repeat the process
        df_weights_by_sector = df[['GICS Sector', 'Weights']].groupby('GICS Sector').sum()
        sectors_above_threshold = df_weights_by_sector.loc[df_weights_by_sector['Weights'] > sector_threshold].index

    return df

In [44]:
sector_threshold = 0.17
weights_after_sector = sector_constraints(df_esg, sector_threshold)

#Check that none of the sectors have a weight above the threshold
#weights_after_sector[['GICS Sector', 'Weights']].groupby('GICS Sector').sum()

#Check that the sum of the weights is equal to 1
#weights_after_sector[['GICS Sector', 'Weights']].groupby('GICS Sector').sum().sum()

weights_after_sector.sort_values(by='Weights', ascending=False)

,Security,GICS Sector,GICS Sub-Industry,ESG Score,Weights
Symbol,,,,,
AMZN,Amazon,Consumer Discretionary,Broadline Retail,29.01,0.034318
NVDA,Nvidia,Information Technology,Semiconductors,12.23,0.033441
AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",16.79,0.029488
COST,Costco,Consumer Staples,Consumer Staples Merchandise Retail,29.12,0.019609
JNJ,Johnson & Johnson,Health Care,Pharmaceuticals,20.10,0.017793
...,...,...,...,...,...
INCY,Incyte,Health Care,Biotechnology,23.71,0.000043
PNR,Pentair,Industrials,Industrial Machinery & Supplies & Components,22.00,0.000043
GNRC,Generac,Industrials,Electrical Components & Equipment,21.93,0.000022


## Stock constraints

In [45]:
def stock_constraints(df, stock_threshold):
    """
    df: DataFrame following the same format as the one returned by sector_constraints
    stock_threshold: float between 0 and 1 that a stock cannot exceed in the portfolio
    """
    #We create a list of the stocks that have a weight above the threshold
    stocks_above_threshold = df.loc[df['Weights'] > stock_threshold].index
    while len(stocks_above_threshold) > 0:
        #we set the weights of the stocks above the threshold to the threshold
        df.loc[stocks_above_threshold, 'Weights'] = stock_threshold
        #we increase (proportionally) the weights of the other stocks to keep the sum of the weights equal to 1
        df.loc[~df.index.isin(stocks_above_threshold), "Weights"] /= df.loc[~df.index.isin(stocks_above_threshold), "Weights"].sum() / (1-stock_threshold*len(stocks_above_threshold))
        #Some stocks may now have a weight above the threshold so we repeat the process
        stocks_above_threshold = df.loc[df['Weights'] > stock_threshold].index

    return df

In [46]:
#print("Max weight for a stock before stock constraints: ", weights_after_sector['Weights'].max())
stock_threshold = 0.05
weights_after_stock = stock_constraints(weights_after_sector, stock_threshold)
#print("Max weight for a stock after stock constraints: ", weights_after_stock['Weights'].max())

#Check that none of the stocks have a weight above the threshold
#weights_after_stock['Weights'].max()

#Check that the sum of the weights is equal to 1
#weights_after_stock['Weights'].sum()

weights_after_stock.sort_values(by='Weights', ascending=False)

,Security,GICS Sector,GICS Sub-Industry,ESG Score,Weights
Symbol,,,,,
AMZN,Amazon,Consumer Discretionary,Broadline Retail,29.01,0.034318
NVDA,Nvidia,Information Technology,Semiconductors,12.23,0.033441
AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",16.79,0.029488
COST,Costco,Consumer Staples,Consumer Staples Merchandise Retail,29.12,0.019609
JNJ,Johnson & Johnson,Health Care,Pharmaceuticals,20.10,0.017793
...,...,...,...,...,...
INCY,Incyte,Health Care,Biotechnology,23.71,0.000043
PNR,Pentair,Industrials,Industrial Machinery & Supplies & Components,22.00,0.000043
GNRC,Generac,Industrials,Electrical Components & Equipment,21.93,0.000022


# References
### Implementations of equal risk contribution
- https://github.com/matthewgilbert/erc/blob/master/erc/erc.py
- https://github.com/mirca/riskparity.py (not used)
- https://thequantmba.wordpress.com/2016/12/14/risk-parityrisk-budgeting-portfolio-in-python/

### Papers
- [Paper of Maillard, Roncalli and Teiletche](http://thierry-roncalli.com/download/erc.pdf)
- [Slides of Maillard, Roncalli and Teiletche](http://www.thierry-roncalli.com/download/erc-slides.pdf)
- [Master's thesis of David Stefanovits](https://ethz.ch/content/dam/ethz/special-interest/math/risklab-dam/documents/walter-saxer-preis/ma-stefanovits.pdf)
